In [27]:
import os
import pandas as pd

In [28]:
BASE_DIR = '/mnt/f/OneDrive/文档（共享）/4_古代DNA/'

In [29]:
# 获取`/mnt/f/OneDrive/文档（共享）/4_古代DNA`文件夹下的所有的文件夹内的所有的命名为`aDNA.txt`文件，将其合并为一个DataFrame
def merge_adna_files(root_dir):
    adna_files = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file == 'aDNA.txt':
                adna_files.append(os.path.join(root, file))

    # 合并所有的文件，并记录来源文件
    df_list = []
    for file in adna_files:
        df = pd.read_csv(file, sep='\t', header=None)
        df['File'] = file  # 新增：记录来源文件
        df_list.append(df)

    merged_df = pd.concat(df_list, ignore_index=True)
    return merged_df

In [30]:
merged_df = merge_adna_files(BASE_DIR)
# 列名命名为`ID`
merged_df.columns = ['ID', 'File']
merged_df
# 获取`ID`列的前缀(即数字之前的字母部分)
merged_df['Prefix'] = merged_df['ID'].str.extract(r'([a-zA-Z]+)')[0]
# 检查前缀有多少种独特的类型
unique_prefixes = merged_df['Prefix'].unique()
print(f"检测到ID中的前缀如下所示: {unique_prefixes}")
print(f"一般来说:`SRR`前缀→NCBI")
print(f"\t`SAMEA/ERR`前缀→ENA")
print(f"如果出现其他前缀，请检查数据源是否正确。")

检测到ID中的前缀如下所示: ['SRR' 'ERR']
一般来说:`SRR`前缀→NCBI
	`SAMEA/ERR`前缀→ENA
如果出现其他前缀，请检查数据源是否正确。


In [31]:
# 查看总共有多少个ID
print(f"总共有{len(merged_df)}个ID")
# 查看有多少重复的ID
duplicate_ids = merged_df[merged_df.duplicated(subset=['ID'], keep=False)]
if not duplicate_ids.empty:
    print(f"检测到{len(duplicate_ids)}个重复的ID:")
    print(duplicate_ids)
else:
    print("没有检测到重复的ID。")
# 查看重复的ID来自哪个文件
duplicate_files = duplicate_ids['File'].unique()
print(f"重复的ID来自以下文件: {duplicate_files}")

总共有911个ID
没有检测到重复的ID。
重复的ID来自以下文件: []


In [32]:
merged_df.drop_duplicates(subset='ID')

,ID,File,Prefix
0,SRR17043543,/mnt/f/OneDrive/文档（共享）/4_古代DNA/ArianoCell2022/...,SRR
1,SRR17043542,/mnt/f/OneDrive/文档（共享）/4_古代DNA/ArianoCell2022/...,SRR
2,SRR17043541,/mnt/f/OneDrive/文档（共享）/4_古代DNA/ArianoCell2022/...,SRR
3,SRR17043540,/mnt/f/OneDrive/文档（共享）/4_古代DNA/ArianoCell2022/...,SRR
4,SRR17043539,/mnt/f/OneDrive/文档（共享）/4_古代DNA/ArianoCell2022/...,SRR
...,...,...,...
906,ERR4332646,/mnt/f/OneDrive/文档（共享）/4_古代DNA/YakaSomelCurrBi...,ERR
907,ERR4332647,/mnt/f/OneDrive/文档（共享）/4_古代DNA/YakaSomelCurrBi...,ERR
908,ERR4332648,/mnt/f/OneDrive/文档（共享）/4_古代DNA/YakaSomelCurrBi...,ERR
909,ERR4332649,/mnt/f/OneDrive/文档（共享）/4_古代DNA/YakaSomelCurrBi...,ERR


In [33]:
# 将不同的前缀的df分别保存到`/mnt/f/OneDrive/文档（共享）/4_古代DNA/`的不同的前缀的txt文件中
for prefix in unique_prefixes:
    prefix_df = merged_df[merged_df['Prefix'] == prefix]
    output_file = f'{BASE_DIR}/{prefix}_aDNA.txt'
    prefix_df[['ID']].to_csv(output_file, sep='\t', index=False, header=False)
    print(f"已将前缀为{prefix}的ID保存到{output_file}")

已将前缀为SRR的ID保存到/mnt/f/OneDrive/文档（共享）/4_古代DNA//SRR_aDNA.txt
已将前缀为ERR的ID保存到/mnt/f/OneDrive/文档（共享）/4_古代DNA//ERR_aDNA.txt
